In [1]:
import altair as alt
import pandas as pd
import panel as pn
from vega_datasets import data
pn.extension('vega')

# Make sure that the Panel version is ≥ 0.14.0.
pn.__version__

'0.14.2'

In [2]:
# from google.colab import drive
# drive=drive.mount('/content/drive')

## Country Map

1.prepare country data

2.create base map, add country data into the base map, combined with country id (create id from code_df)

3.add year slider

4.add movie and show selection

In [3]:
basedir='/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/Data Visualization/assignment 3/DV/raw_titles.csv'

data1 = pd.read_csv(basedir)
data1.drop(columns = 'index', inplace = True)
data1.sort_values('release_year', inplace = True)
# Filter data to keep rows with non-null values these 3 columns
data1 = data1.dropna(subset=['production_countries', 'type','release_year'])

# the first year a country produced Netflix content, the content name and the according imdb_score
data1 = data1.assign(production_countries=data1['production_countries'].str.split(','))
data1 = data1.explode('production_countries')
data1['production_countries'] = data1['production_countries'].str.replace('[^a-zA-Z]', '')
year_country = pd.concat([data1.release_year,data1.production_countries,data1.type],axis=1)
countryexplode = year_country.explode("production_countries")

countryeplode_num =countryexplode.groupby(['production_countries','release_year','type'])["production_countries"]\
          .agg([('num_productions','count')]).reset_index()\
          .sort_values(by=["release_year",'production_countries'] , ascending=[True,True])

countryeplode_num =  countryeplode_num.drop(index =countryeplode_num[(countryeplode_num.production_countries == '')].index.tolist() )
countryeplode_num.reset_index(inplace = True,drop = True)

# Group data by 'production_countries' and 'release_year' columns and count the number of shows in each group
country_year_counts = countryeplode_num.groupby(['production_countries','release_year','type'])['num_productions'].agg('sum').reset_index(name='Count')
country_year_counts.columns = ['Country','Year','Type','Count']

country_year_counts[country_year_counts['Country']=='US']

#prepare data for base map
world = data.world_110m.url
world_topo = data.world_110m()
world_topo.keys()
world_topo['objects'].keys()

#prepare Country Codes
code_df = pd.read_json(
    "https://raw.githubusercontent.com/alisle/world-110m-country-codes/master/world-110m-country-codes.json"
)
# Add a missing country
singapore_df = pd.DataFrame(data=[{"code": "Singapore", "id": 702, "name": "Singapore"}])
code_df = pd.concat([code_df, singapore_df])

# Merge netflix country data with country code
code_df2 = code_df.set_index("code")
country_year_counts2 = country_year_counts.set_index("Country")
df = country_year_counts2.join(code_df2).reset_index()

base_map=alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape( stroke='#d8e3e7', 
                                  strokeWidth=0.3,
                                  color='#f9f9f9'
                                  # color='#d8e3e7'
                                  )
# base_map

count_max = df['Count'].max()
count_min = df['Count'].min()
chart = alt.layer(
    base_map,

    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape( 
    stroke='#FFFFF', 
    strokeWidth=0.25
    ).transform_lookup(
        lookup='id', from_=alt.LookupData(data=df, key='id', fields=['Count','Year','Type','index','name'])
    ).encode(
        alt.Color('Count:Q',
                  scale=alt.Scale(clamp=True,domain=[count_min, count_max]), 
                  legend=alt.Legend()),
        tooltip=['name:N', 'Year:N','Count:Q','Type:N'],
    ).project(
        type='equalEarth'
    )
    ).properties(
        width=1000,
        height=500
    ).configure_view(
        stroke=None
    )


type_selector = pn.widgets.Select( value='type', width=150, options=['MOVIE', 'SHOW'])
year_slider = pn.widgets.IntSlider(name='Year',width=850, start=int( 2000
                                # df['Year'].min()
                                ),
                                   end=int(df['Year'].max()), step=1, value=int(df['Year'].max()))


def update_chart(event):
    selected_type = type_selector.value
    selected_year = year_slider.value
    filtered_df = df[(df['Year'] == selected_year) & (df['Type'] == selected_type)]


    
    # filtered data
    chart = alt.layer(
        base_map,
        alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape().transform_lookup(
            lookup='id', from_=alt.LookupData(data=filtered_df, key='id', fields=['Count', 'Year', 'Type', 'index', 'name'])
        ).encode(
            
            alt.Color('Count:Q',
                      scale=alt.Scale(clamp=True,domain=[count_min, count_max]), 
                      legend=alt.Legend()),
            tooltip=['name:N', 'Year:N', 'Count:Q', 'Type:N'],
            
        ).project(
            type='equalEarth'
        )
    ).properties(
        width=1000,
        height=500
    ).configure_view(
        stroke=None
    )
    
    # Update the chart in the Panel layout
    layout1[2] = chart


# Attach the callback function
type_selector.param.watch(update_chart, 'value')
year_slider.param.watch(update_chart, 'value')

# Create a panel layout with the selector and the chart
layout1 = pn.Column(
    '## Netflix by Year and Country',
    pn.Row(type_selector,
    year_slider),
    chart,
)
layout1

/var/folders/jn/gt3868vd039by14gxmgb54b40000gn/T/ipykernel_84452/3581348897.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data1['production_countries'] = data1['production_countries'].str.replace('[^a-zA-Z]', '')


BokehModel(combine_events=True, render_bundle={'docs_json': {'c5ac28b5-5aa2-4061-ac63-e0db9f6589ef': {'defs': …

Column
    [0] Markdown(str)
    [1] Row
        [0] Select(options=['MOVIE', 'SHOW'], value='MOVIE', width=150)
        [1] IntSlider(end=2022, name='Year', start=2000, value=2022, width=850)
    [2] Vega(LayerChart, selection=Selection)

## Genre

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from altair import expr, datum
import numpy as np
import panel as pn
import pandas as pd
import altair as alt

# prepare data
df3 = pd.read_csv('/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/Data Visualization/assignment 3/DV/raw_titles.csv')
df3 = df3.set_index('index')


df_alt = df3.assign(production_countries=df3['production_countries'].str.split(','))
df_alt = df_alt.explode('production_countries')
df_alt['production_countries'] = df_alt['production_countries'].str.replace('[^a-zA-Z]', '')

df_alt = df_alt.assign(genres=df_alt['genres'].str.split(','))
df_alt = df_alt.explode('genres')
df_alt['genres'] = df_alt['genres'].str.replace('[^,a-zA-Z]', '')

df_alt = df_alt[0:15000]

df4 = df_alt.drop_duplicates(subset='id', keep='first')
df4 = df4.sort_values('imdb_votes', ascending=False).reset_index(drop=True)
df4 = df4.reset_index(drop=False)
df4 = df4[['index','id','title','type','genres','imdb_score','imdb_votes']]

color = alt.Color('genres:N')

# # Create multiple selection on the bar chart
# multi = alt.selection_multi(name = 'multi',fields=['genres'])
# # Create interval selecion on the bubble chart
# brush = alt.selection_interval(name = 'brush',encodings = ['x'])

# # Create a bubble chart
# points = alt.Chart(df4).mark_point().encode(
#     alt.X('index:Q', title='movie/show'),
#     alt.Y('imdb_score:Q',
#         title='imdb_score',
#         scale=alt.Scale(domain=[-1, 10])
#     ),
#     color=alt.condition(multi,color,alt.value('lightgrey'))
# ).properties(
#     width=400,
#     height=310
# ).add_selection(brush
#                 ).transform_filter(
#     alt.FieldOneOfPredicate(field='type', oneOf=['MOVIE'])  # Filter data for 'Action' and 'Comedy' genres
# )

# # Filter the data when user clicks on the bar chart

# # Add interval selection to the bubble chart


# # Create a bar chart
# bars = alt.Chart(df4).mark_bar().encode(
#     alt.X('mean(imdb_votes):Q',title='average votes number of each genre '),
#     alt.Y('genres:N'),
#     color=alt.condition(multi,color,alt.value('lightgrey'))
# ).properties(width=400,height=310).add_selection(multi
#                                       ).transform_filter(
#     alt.FieldOneOfPredicate(field='type', oneOf=['MOVIE'])  # Filter data for 'Action' and 'Comedy' genres
# )

# # Add multiple selection to the bar chart


# # Filter the data when user draws brush on the bubble chart
# bars = bars.transform_filter(brush)

# points & bars

"""## panel"""

# def filtered_table(selection,dataframe):
#     # 0. Prevent an error if selection object is None or empty.
#     if selection is None or len(selection) == 0:
#         return '### No data selection now' 
    
#     # 1. Create a query string to filter the data
#     query = ' & '.join(
#         f'{crange[0]} <= `{col}` <= {crange[1]}'
#         for col, crange in selection.items())
    
#     # 2. Filter the penguin data frame from the query
#     df_filter = dataframe.query(query)

    
#     # 3. Add the DataFrame pane that render pandas object
#     return pn.pane.DataFrame(df_filter, width=400)



pn.extension('vega')


def plot_scatter(dataframe):
    # 1. Copy the scatter plot and modify encodings based on params
    brush = alt.selection_interval(name = 'brush',encodings = ['x'])
    multi = alt.selection_multi(name='multi', fields=['genres'])
    points = alt.Chart(dataframe).mark_point().encode(
        alt.X('index:Q', title='movie/show'),
        alt.Y('imdb_score:Q',
            title='imdb_score',
            scale=alt.Scale(domain=[-1, 10])
        ),
        color=alt.condition(brush,color,alt.value('lightgrey'))
    ).properties(
        width=400,
        height=310
    ).add_selection(brush)

    # 2. Retuen the Vega pane of the Altair chart
    return pn.pane.Vega(points, debounce=100)

def plot_bar(selection,dataframe):
    # 1. Filter the penguin data frame based on selection
    if selection is None or len(selection) == 0:
        df_plot_bar = dataframe
    else:
        query = ' & '.join(
            f'{crange[0]} <= `{col}` <= {crange[1]}'
            for col, crange in selection.items())
        df_plot_bar = dataframe.query(query)

    # 2. Create the bar chart
    multi = alt.selection_multi(name = 'multi',fields=['genres'])

    brush = alt.selection_interval(name = 'brush',encodings = ['x'])

    bars = alt.Chart(df_plot_bar).mark_bar().encode(
        x='mean(imdb_votes):Q',
        y='genres:N',
        color=alt.condition(multi,color,alt.value('lightgrey')),
        tooltip=['mean(imdb_votes):Q', 'genres:N','Type:N'],
    ).properties(width=400, height=310).add_selection(multi)
    # Add multiple selection to the bar chart


    # Filter the data when user draws brush on the bubble chart
    # bars = bars.transform_filter(brush)
    return bars


# movie/show selection
type = pn.widgets.Select(value='type', width=150, options=['MOVIE', 'SHOW'])


pane = plot_scatter(df4)

# 3. Update the panel when the inputs are changed
def update(event):
    select_type = type.value
    filtered_df = df4[(df4['type'] == select_type)]
    pane = plot_scatter(filtered_df)
    layout2[2][0][0] = pane
    layout2[2][0][1] = pn.bind(plot_bar, pane.selection.param.brush,filtered_df)

# 2. Watch the parameter change and call update function

type.param.watch(update, 'value')


# 4. Create the panel with inputs and all panes

layout2 = pn.Column(
    '## Netflix Number of Votes Explorer',
    type,
    pn.Column(
        pn.Row(pane, pn.bind(plot_bar, pane.selection.param.brush,df4))
        )
)



layout2

/var/folders/jn/gt3868vd039by14gxmgb54b40000gn/T/ipykernel_84452/3245111079.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df_alt['production_countries'] = df_alt['production_countries'].str.replace('[^a-zA-Z]', '')
/var/folders/jn/gt3868vd039by14gxmgb54b40000gn/T/ipykernel_84452/3245111079.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df_alt['genres'] = df_alt['genres'].str.replace('[^,a-zA-Z]', '')


BokehModel(combine_events=True, render_bundle={'docs_json': {'b01ab5f0-e5e1-4a6c-94a1-7e1863072fbe': {'defs': …

Column
    [0] Markdown(str)
    [1] Select(options=['MOVIE', 'SHOW'], value='MOVIE', width=150)
    [2] Column
        [0] Row
            [0] Vega(Chart, debounce=100, selection=Selection)
            [1] ParamFunction(function)

## Imdb Score

In [5]:
"""
1.   Load dataset
2.   import Altair
3. plot
"""

# max row > 5000
alt.data_transformers.disable_max_rows()

# show
# Load dataset
show = pd.read_csv('/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/Data Visualization/assignment 3/DV/show.csv')
movie = pd.read_csv('/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/Data Visualization/assignment 3/DV/movie.csv')

show = show.loc[:, ['id', 'title', 'type','genre_split', 'imdb_score']]
source_show = show.rename(columns={'genre_split': 'genre'})

movie = movie.loc[:, ['id', 'title', 'type','genre_split', 'imdb_score']]
source_movie = movie.rename(columns={'genre_split': 'genre'})

# exclude null
source_show = source_show.dropna(subset=['genre'])
source_movie = source_movie.dropna(subset=['genre'])

# plot - show
scat_show = alt.Chart(source_show).mark_point(size = 50).encode(
    y = alt.Y('mean(imdb_score):Q', scale=alt.Scale(domain=(4,8)), title='Average Imdb Score'),
    x = alt.X('count(id)', title='Count of show'),
    color = 'genre',
    size = alt.Size('count(id)'),
    tooltip = ['genre:N', 'mean(imdb_score):N']
).properties(
    width=400,
    height=400
)


scat_movie = alt.Chart(source_movie).mark_point(size = 50).encode(
    y = alt.Y('mean(imdb_score):Q', scale=alt.Scale(domain=(4,8)),title='Average Imdb Score'),
    x = alt.X('count(id)', title='Count of movie'),
    color = 'genre',
    size = alt.Size('count(id)'),
    tooltip = ['genre:N', 'mean(imdb_score):N']
).properties(
    width=400,
    height=400
)

layout3=pn.Column(
    '## Netflix Imdb Score',
    pn.Row(scat_movie,
    scat_show)
    )
layout3

BokehModel(combine_events=True, render_bundle={'docs_json': {'eda06358-d3eb-4d48-81dc-70e282d33e72': {'defs': …

Column
    [0] Markdown(str)
    [1] Row
        [0] Vega(Chart, selection=Selection)
        [1] Vega(Chart, selection=Selection)

## Actor

In [6]:
import altair as alt
import pandas as pd
import panel as pn
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from altair.expr import datum

pn.extension("material")  # apply the 'elegant' theme

# max row > 5000
alt.data_transformers.disable_max_rows()

# Load dataset
df_actor_info = pd.read_csv('/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/Data Visualization/assignment 3/DV/df_actor_info.csv')
df_actor_detail = pd.read_csv('/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/Data Visualization/assignment 3/DV/df_actor_detail.csv')
sorted_cooperation_df = pd.read_csv('/Users/flickr-xc/Library/Mobile Documents/com~apple~CloudDocs/DSBA Courses/Data Visualization/assignment 3/DV/sorted_cooperation_df.csv')

# define the widgets
name_input = pn.widgets.AutocompleteInput(name='Name', options=list(df_actor_info['name'].unique()), case_sensitive=False, placeholder='Enter name...')
type_input = pn.widgets.Select(name='Type', options=['movie', 'show'], value='movie', width=200)

# define the function that updates the plot
def update_plot(name, type):
    filtered_df = df_actor_info[(df_actor_info['name'].str.lower() == name.lower()) & (df_actor_info['type'].str.lower() == type)]

    if not filtered_df.empty:
        total_votes = int(filtered_df['total_votes'].iloc[0])
        average_score = round(filtered_df['average_score'].iloc[0], 2)
        total_numbers = filtered_df['total_numbers'].iloc[0]

        total_votes_pane = pn.pane.HTML(
          f'<h2 style="color:#0077C0;font-size:40px;font-family: Verdana">{total_votes:,}</h2><br><h4 style="color:#0077C0;font-family: Verdana">Total Votes</h4>',
          margin=(0, 0, 20, 0))
        average_score_pane = pn.pane.HTML(
          f'<h2 style="color:#FFB000;font-size:40px;font-family: Verdana">{average_score}</h2><br><h4 style="color:#FFB000;font-family: Verdana">Average Score</h4>',
          margin=(0, 20, 20, 0))
        total_numbers_pane = pn.pane.HTML(
          f'<h2 style="color:#DC267F;font-size:40px;font-family: Verdana">{total_numbers}</h2><br><h4 style="color:#DC267F;font-family: Verdana">Number of Works</h4>',
          margin=(0, 20, 20, 0))

        result = pn.Row(pn.Column(total_votes_pane, average_score_pane, total_numbers_pane),
                        sizing_mode='stretch_both', margin=(0, 20))

        return result
        
    else:
        return pn.pane.HTML('<h3>No data found.</h3>')

# define the function that updates the table
def update_table(name, type):
    filtered_df = df_actor_detail[(df_actor_detail['name'].str.lower() == name.lower()) & (df_actor_detail['type'].str.lower() == type)]

    if not filtered_df.empty:
        table_rows = []
        for index, row in filtered_df.iterrows():
            title = row['title']
            type = row['type'].capitalize()
            genres = row['genres']
            release_year = row['release_year']
            imdb_score = row['imdb_score']
            imdb_votes = int(row['imdb_votes'])
            row_html = f"""
            <div style="background-color:#F0F8FF;border: 2px solid #E65100;border-radius:5px;padding:10px;margin-bottom:10px">
                <h3 style="color:#E65100;font-family: Verdana">{title}</h3>
                <table style="margin-left:10px">
                    <tr>
                        <td style="font-family: Verdana;font-size:10px"><b>Type:</b></td>
                        <td style="font-family: Verdana;font-size:10px">{type}</td>
                    </tr>
                    <tr>
                        <td style="font-family: Verdana;font-size:10px"><b>Genre:</b></td>
                        <td style="font-family: Verdana;font-size:10px">{genres}</td>
                    </tr>
                    <tr>
                        <td style="font-family: Verdana;font-size:10px"><b>Year of Release:</b></td>
                        <td style="font-family: Verdana;font-size:10px">{release_year}</td>
                    </tr>
                    <tr>
                        <td style="font-family: Verdana;font-size:10px"><b>IMDB Score:</b></td>
                        <td style="font-family: Verdana;font-size:10px">{imdb_score}</td>
                    </tr>
                    <tr>
                        <td style="font-family: Verdana;font-size:10px"><b>IMDB Votes:</b></td>
                        <td style="font-family: Verdana;font-size:10px">{imdb_votes:,}</td>
                    </tr>
                </table>
            </div>
            """

            table_rows.append(row_html)

        result = pn.Column(*table_rows, sizing_mode='stretch_both', margin=(0, 20))
        return result
        
    else:
        return pn.pane.HTML('<h3>No data found.</h3>')


# Set the default theme for Altair charts
alt.themes.enable('fivethirtyeight')
# Define a custom CSS style for the text pane
text_style = 'font-family: Arial; font-size: 14px; color: #333; margin: 5px 0;'

def update_cooperate(name, type):
    filtered_df = sorted_cooperation_df[(sorted_cooperation_df['name_x'].str.lower() == name.lower())]

    if not filtered_df.empty:
        # Filter the top 10 partners with the largest number of cooperations
        df_top_10 = filtered_df.nlargest(20, 'cooperations')

        # Define the color scheme for the bars
        color_scale = alt.Scale(range=['#E50914', '#F5F5F5', '#B81D24', '#D81E26', '#F0343F', '#E50914', '#F9AA33', '#F5F5F5', '#B81D24', '#D81E26']
)

        # define x_axis
        x_axis = alt.Axis(
        title='Number of Cooperations',
        labelExpr="datum.label == floor(datum.label) ? format(datum.label, 'd') : ''",
        grid=False,
        tickCount=6
        )
        # Create the bar chart using Altair
        bars = alt.Chart(df_top_10).mark_bar(size=15, opacity=0.7).encode(
            x=alt.X('cooperations', title='Number of Cooperations', axis=x_axis),
            y=alt.Y('name_y', title='Cooperation Partner', sort=alt.EncodingSortField(field='cooperations', op='sum', order='descending')),
            color=alt.Color('cooperations', scale=color_scale, legend=None),
            tooltip=[alt.Tooltip('name_y:N', title='Partner Name'), alt.Tooltip('cooperations', title='Number of Cooperations')]
        ).properties(
            height=400,
            width=600,
            title=f'Cooperation Partners of {name.title()}',
            background='#f5f5f5'
        ).interactive()

        text_pane = pn.pane.HTML(margin=(20, 10), style={'background-color': '#f9f9f9', 'border': '1px solid #cccccc', 'border-radius': '5px', 'padding': '10px'})

        # Create the layout and return it
        layout = pn.Row(bars, text_pane, margin=(20, 20), background='#f9f9f9')
        return layout
        
    else:
        return pn.pane.HTML('<h3>No data found.</h3>')

    
# define the panel app
@pn.depends(name_input.param.value, type_input.param.value)
def panel_app(name, type):
    if name and type:
        # create the two panes side by side using Tabs
        return pn.Tabs(('Overview', update_plot(name, type)), \
                       ('Into Detail', update_table(name, type)),\
                       ('Best Partners',update_cooperate(name, type)))
    else:
        return pn.pane.HTML('<h3>Please select name and type.</h3>')


# show the panel app
layout4 =pn.Column(
    '## Actors On Netflix',
    pn.Row(name_input, type_input),
    panel_app
)

layout4

BokehModel(combine_events=True, render_bundle={'docs_json': {'c4cc9f3f-617f-4fa0-a997-85b81bea1eab': {'defs': …

Column
    [0] Markdown(str)
    [1] Row
        [0] AutocompleteInput(case_sensitive=False, name='Name', options=['Tamsin Greig', ...], placeholder='Enter name...')
        [1] Select(name='Type', options=['movie', 'show'], value='movie', width=200)
    [2] ParamFunction(function)

## Final Layout

In [7]:
layout_final = pn.Column(
    '# Netflix',
    layout1,
    layout2,
    layout3,
    layout4
    )

layout_final.servable()



BokehModel(combine_events=True, render_bundle={'docs_json': {'e96fb7ba-52bb-4369-8481-c1b36a6c01f1': {'defs': …

Column
    [0] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] Select(options=['MOVIE', 'SHOW'], value='MOVIE', width=150)
            [1] IntSlider(end=2022, name='Year', start=2000, value=2020, width=850)
        [2] Vega(LayerChart, selection=Selection)
    [2] Column
        [0] Markdown(str)
        [1] Select(options=['MOVIE', 'SHOW'], value='MOVIE', width=150)
        [2] Column
            [0] Row
                [0] Vega(Chart, debounce=100, selection=Selection)
                [1] ParamFunction(function)
    [3] Column
        [0] Markdown(str)
        [1] Row
            [0] Vega(Chart, selection=Selection)
            [1] Vega(Chart, selection=Selection)
    [4] Column
        [0] Markdown(str)
        [1] Row
            [0] AutocompleteInput(case_sensitive=False, name='Name', options=['Tamsin Greig', ...], placeholder='Enter name...', value='Brad Pitt', value_input='brad p')
            [1] Select(name='Type', options=['movie', 'show'], value='movie', width=200)
        [2] ParamFunction(function)

In [8]:

# Save the app as an HTML file
layout_final.save("Netflix_app.html")
